# 🧠 Legal Assistant – Jordanian Labour Law (EN + AR) Extraction Notebook

This notebook prepares **two parallel datasets** for the Jordanian Labour Law (Law No. 8 of 1996):

- **English**: the official Ministry/Official Gazette English PDF.
- **Arabic**: your **Google-Docs–cleaned OCR Arabic** PDF.

It extracts text, splits into articles, normalizes metadata, chunks the articles, and writes:
- `labour_law_articles_en.json`
- `labour_law_articles_ar.json`
- `labour_law_articles_multilingual.json`
- `labour_law_chunks_multilingual.json`
- `sanity_report_multilingual.json`

> ⚠️ Disclaimer: This dataset is for informational/educational use and is **not** legal advice.


In [18]:
from pathlib import Path
from PyPDF2 import PdfReader
import re
import json
import unicodedata
from datetime import datetime


In [19]:
BASE_DIR = Path(r"C:\Users\VICTUS-H\Desktop\Faris\Datasets\raw")

PDF_EN_PATH = BASE_DIR / "jordanian_labour_law_no._(8)_of_1996_and_its_amendments.pdf"
PDF_AR_PATH = BASE_DIR / "(قانون العمل لسنة 1996) .pdf"

OUT_DIR = BASE_DIR
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("EN PDF exists:", PDF_EN_PATH.exists(), PDF_EN_PATH)
print("AR PDF exists:", PDF_AR_PATH.exists(), PDF_AR_PATH)


EN PDF exists: True C:\Users\VICTUS-H\Desktop\Faris\Datasets\raw\jordanian_labour_law_no._(8)_of_1996_and_its_amendments.pdf
AR PDF exists: True C:\Users\VICTUS-H\Desktop\Faris\Datasets\raw\(قانون العمل لسنة 1996) .pdf


In [20]:
def extract_pdf_text(pdf_path: Path) -> str:
    reader = PdfReader(str(pdf_path))
    parts = []
    for page in reader.pages:
        t = page.extract_text()
        if t:
            parts.append(t)
    return "\n".join(parts)


full_text_en = extract_pdf_text(PDF_EN_PATH)
full_text_ar = extract_pdf_text(PDF_AR_PATH)

print("EN TEXT LENGTH:", len(full_text_en))
print("AR TEXT LENGTH:", len(full_text_ar))

print("\nEN preview:\n", full_text_en[:500])
print("\nAR preview:\n", full_text_ar[:500])


EN TEXT LENGTH: 118963
AR TEXT LENGTH: 104296

EN preview:
 1 
 Jordanian Labour Law No. (8) Of 1996 and its Amendments 
 
 
CHAPTER ONE  
Scope of Application of this Law  and Interpretation of Terms  
 
Article (1):  Title and Effective Date  
This Law shall be cited as the "The labour code for the year 1996" and shall come 
into effect after sixty days of its publication in the Official Gazette.  
 
Article (2):  Definitions Wherever used in this Law, and unless the context 
otherwise provides, the following terms and expressions, shall have the mean

AR preview:
 اﻟﻣﺎدة
 
1
   
ﯾﺳﻣﻰ
 
ھذا
 
اﻟﻘﺎﻧون
 
)ﻗﺎﻧون
 
اﻟﻌﻣل
 
ﻟﺳﻧﺔ
 
1996
(
 
وﯾﻌﻣل
 
ﺑﮫ
 
ﺑﻌد
 
ﻣرور
 
ﺳﺗﯾن
 
ﯾوﻣﺎ
 
ﻋﻠﻰ
 
ﺗﺎرﯾﺦ
 
ﻧﺷره
 
ﻓﻲ
 
اﻟﺟرﯾدة
 
اﻟرﺳﻣﯾﺔ.
 
 
 
اﻟﻣﺎدة
 
2
 
ﯾﻛون
 
ﻟﻠﻛﻠﻣﺎت
 
واﻟﻌﺑﺎرات
 
اﻟﺗﺎﻟﯾﺔ
 
ﺣﯾﺛﻣﺎ
 
وردت
 
ﻓﻲ
 
ھذا
 
اﻟﻘﺎﻧون
 
اﻟﻣﻌﺎﻧﻲ
 
اﻟﻣﺧﺻﺻﺔ
 
ﻟﮭﺎ
 
أدﻧﺎه
 
ﻣﺎ
 
ﻟم
 
ﺗدل
 
اﻟﻘرﯾﻧﺔ
 
ﻋﻠﻰ
 
ﻏﯾر
 
ذﻟك:
 
 
 
اﻟوزارة:
 
وزارة
 
اﻟﻌﻣل.
 
 
 
اﻟوزﯾر:
 
وزﯾر
 
اﻟﻌﻣل.
 
 
 
اﻷﻣﯾن
 
اﻟﻌﺎم:
 
اﻷﻣﯾن
 
اﻟﻌﺎم

In [21]:
def normalize_arabic_text(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)

    arabic_digit_map = str.maketrans("٠١٢٣٤٥٦٧٨٩", "0123456789")
    text = text.translate(arabic_digit_map)

    text = re.sub(r"\r", "\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)

    text = re.sub(r"(المادة)\s*\n\s*(\d{1,3})", r"\1 \2", text)

    return text


full_text_ar = normalize_arabic_text(full_text_ar)


In [22]:
def split_articles_en(full_text: str):
    pattern = re.compile(
        r"\bArticle\s*\(?\s*(\d{1,3})\s*\)?\s*[:\-]?",
        re.IGNORECASE
    )

    matches = list(pattern.finditer(full_text))
    articles = []

    for idx, m in enumerate(matches):
        art_no = int(m.group(1))
        start = m.start()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(full_text)
        raw = full_text[start:end].strip()
        raw = pattern.sub("", raw, count=1).strip()
        articles.append({"article_number": art_no, "text": raw})

    return articles


def split_articles_ar(full_text: str):
    pattern = re.compile(
        r"(المادة\s*(?:\(|\[)?\s*(\d{1,3})\s*(?:\)|\])?)",
        re.UNICODE
    )

    matches = list(pattern.finditer(full_text))
    articles = []

    for idx, m in enumerate(matches):
        art_no = int(m.group(2))
        start = m.start()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(full_text)
        raw = full_text[start:end].strip()
        articles.append({"article_number": art_no, "text": raw})

    return articles


articles_en = split_articles_en(full_text_en)
articles_ar = split_articles_ar(full_text_ar)

print("EN articles:", len(articles_en), articles_en[0]["article_number"], articles_en[-1]["article_number"])
print("AR articles:", len(articles_ar), articles_ar[0]["article_number"], articles_ar[-1]["article_number"])


EN articles: 156 1 142
AR articles: 142 1 142


In [23]:
with open(OUT_DIR / "labour_law_articles_en.json", "w", encoding="utf-8") as f:
    json.dump(articles_en, f, ensure_ascii=False, indent=2)

with open(OUT_DIR / "labour_law_articles_ar.json", "w", encoding="utf-8") as f:
    json.dump(articles_ar, f, ensure_ascii=False, indent=2)


In [24]:
en_map = {a["article_number"]: a["text"] for a in articles_en}
ar_map = {a["article_number"]: a["text"] for a in articles_ar}

all_numbers = sorted(set(en_map.keys()) | set(ar_map.keys()))

multilingual_articles = []

for num in all_numbers:
    multilingual_articles.append({
        "article_number": num,
        "en": en_map.get(num),
        "ar": ar_map.get(num)
    })

with open(OUT_DIR / "labour_law_articles_multilingual.json", "w", encoding="utf-8") as f:
    json.dump(multilingual_articles, f, ensure_ascii=False, indent=2)


In [25]:
def chunk_text(text, max_chars=800):
    if not text:
        return []
    chunks = []
    current = ""
    for line in text.split("\n"):
        if len(current) + len(line) > max_chars:
            chunks.append(current.strip())
            current = ""
        current += line + "\n"
    if current.strip():
        chunks.append(current.strip())
    return chunks


chunks = []

for art in multilingual_articles:
    for lang in ("en", "ar"):
        text = art.get(lang)
        for idx, chunk in enumerate(chunk_text(text)):
            chunks.append({
                "article_number": art["article_number"],
                "lang": lang,
                "chunk_id": idx,
                "text": chunk
            })

with open(OUT_DIR / "labour_law_chunks_multilingual.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)


In [26]:
def article_stats(articles):
    nums = [a["article_number"] for a in articles]
    return {
        "count": len(nums),
        "min": min(nums) if nums else None,
        "max": max(nums) if nums else None,
        "missing": sorted(set(range(min(nums), max(nums)+1)) - set(nums)) if nums else []
    }


report = {
    "generated_at": datetime.utcnow().isoformat(),
    "english": article_stats(articles_en),
    "arabic": article_stats(articles_ar)
}

with open(OUT_DIR / "sanity_report_multilingual.json", "w", encoding="utf-8") as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

report


{'generated_at': '2026-02-07T01:00:38.112928',
 'english': {'count': 156, 'min': 1, 'max': 142, 'missing': [18]},
 'arabic': {'count': 142, 'min': 1, 'max': 142, 'missing': []}}

In [27]:
import json
import re
import unicodedata
from pathlib import Path

# =========================
# CONFIG
# =========================
BASE_DIR = Path(r"C:\Users\VICTUS-H\Desktop\Faris\Datasets\raw")

AR_INPUT_JSON = BASE_DIR / "labour_law_articles_ar.json"
AR_OUTPUT_JSON = BASE_DIR / "labour_law_articles_ar_cleaned.json"
MULTI_INPUT_JSON = BASE_DIR / "labour_law_articles_multilingual.json"
MULTI_OUTPUT_JSON = BASE_DIR / "labour_law_articles_multilingual_cleaned.json"
CHUNKS_INPUT_JSON = BASE_DIR / "labour_law_chunks_multilingual.json"
CHUNKS_OUTPUT_JSON = BASE_DIR / "labour_law_chunks_multilingual_cleaned.json"


# =========================
# Arabic Text Cleaning
# =========================
def clean_arabic_text(text: str) -> str:
    if not text:
        return text

    # Unicode normalization (fix presentation forms)
    text = unicodedata.normalize("NFKC", text)

    # Remove tatweel
    text = re.sub("ـ+", "", text)

    # Normalize whitespace
    text = re.sub(r"\r", "\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)

    # Merge broken OCR lines (word-per-line → sentence)
    lines = [l.strip() for l in text.split("\n") if l.strip()]
    merged = []
    buffer = ""

    for line in lines:
        # keep article headers intact
        if line.startswith("المادة"):
            if buffer:
                merged.append(buffer.strip())
                buffer = ""
            merged.append(line)
            continue

        # new paragraph if line ends with punctuation
        if re.search(r"[.:؛؟!]\s*$", line):
            buffer += (" " if buffer else "") + line
            merged.append(buffer.strip())
            buffer = ""
        else:
            buffer += (" " if buffer else "") + line

    if buffer:
        merged.append(buffer.strip())

    text = "\n".join(merged)

    return text


# =========================
# Load & Clean Arabic Articles
# =========================
with open(AR_INPUT_JSON, "r", encoding="utf-8") as f:
    articles_ar = json.load(f)

for art in articles_ar:
    art["text"] = clean_arabic_text(art["text"])

with open(AR_OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(articles_ar, f, ensure_ascii=False, indent=2)


# =========================
# Update Multilingual Articles
# =========================
with open(MULTI_INPUT_JSON, "r", encoding="utf-8") as f:
    multi_articles = json.load(f)

ar_map = {a["article_number"]: a["text"] for a in articles_ar}

for art in multi_articles:
    if art.get("ar"):
        art["ar"] = ar_map.get(art["article_number"], art["ar"])

with open(MULTI_OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(multi_articles, f, ensure_ascii=False, indent=2)


# =========================
# Update Chunks (Arabic Only)
# =========================
def chunk_text(text, max_chars=800):
    if not text:
        return []
    chunks = []
    current = ""
    for line in text.split("\n"):
        if len(current) + len(line) > max_chars:
            chunks.append(current.strip())
            current = ""
        current += line + " "
    if current.strip():
        chunks.append(current.strip())
    return chunks


with open(CHUNKS_INPUT_JSON, "r", encoding="utf-8") as f:
    chunks = json.load(f)

new_chunks = []
for ch in chunks:
    if ch["lang"] == "ar":
        cleaned = clean_arabic_text(ch["text"])
        for idx, txt in enumerate(chunk_text(cleaned)):
            new_chunks.append({
                "article_number": ch["article_number"],
                "lang": "ar",
                "chunk_id": idx,
                "text": txt
            })
    else:
        new_chunks.append(ch)

with open(CHUNKS_OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(new_chunks, f, ensure_ascii=False, indent=2)


print("✅ Arabic cleaning complete")
print("→", AR_OUTPUT_JSON.name)
print("→", MULTI_OUTPUT_JSON.name)
print("→", CHUNKS_OUTPUT_JSON.name)


✅ Arabic cleaning complete
→ labour_law_articles_ar_cleaned.json
→ labour_law_articles_multilingual_cleaned.json
→ labour_law_chunks_multilingual_cleaned.json
